In [1]:
import pandas as pd

f = pd.read_csv("./data.csv", encoding = "cp949")
f

,번호,전체 청구항
0,US9276168,1. A component comprising a substrate comprisi...
1,US9276048,"1. A method of detecting infrared (IR), compri..."
2,US9274264,"1. A light source module, comprising:a light g..."
3,US9273851,"1. A backlight module, which comprises:a diffu..."
4,US9260655,1. A composition comprising:(a) a quantum dot ...
...,...,...
995,US20020063868A1,1. A method for detecting features of an objec...
996,US20020041736A1,What is claimed is: 1. An optical dispersion c...
997,US20020021003A1,1. An anti-counterfeit device comprising quant...
998,US20010031564A1,What is claimed is: 1. An apparatus for fabric...


In [21]:
import re

lst = []

for i in f.iloc[:,1]:
    a = re.sub('[|0-9.,:()|]+', '', i)
    a = re.sub('[|  |]+', ' ', a)
    lst.append(a)
f["전처리"] = lst
f['전처리'] = f['전처리'].apply(lambda x: x.lower())
f


,번호,전체 청구항,전처리
0,US9276168,1. A component comprising a substrate comprisi...,a component comprising a substrate comprising...
1,US9276048,"1. A method of detecting infrared (IR), compri...",a method of detecting infrared ir comprisingp...
2,US9274264,"1. A light source module, comprising:a light g...",a light source module comprisinga light guide...
3,US9273851,"1. A backlight module, which comprises:a diffu...",a backlight module which comprisesa diffuser ...
4,US9260655,1. A composition comprising:(a) a quantum dot ...,a composition comprisinga a quantum dot bindi...
...,...,...,...
995,US20020063868A1,1. A method for detecting features of an objec...,a method for detecting features of an object ...
996,US20020041736A1,What is claimed is: 1. An optical dispersion c...,what is claimed is an optical dispersion compe...
997,US20020021003A1,1. An anti-counterfeit device comprising quant...,an anti-counterfeit device comprising quantum...
998,US20010031564A1,What is claimed is: 1. An apparatus for fabric...,what is claimed is an apparatus for fabricatin...


In [22]:
import csv
from glob import glob
import pandas as pd
import time


processed_data = []
for i in f.iloc[:,2]:
    txt=[]
    k = i.split(" ")
    for j in k:
        txt.append(j)
    txt.append("\n")
    processed_data.append(txt)




In [7]:
file = open("test.txt","w", encoding='utf-8')
for i in processed_data:
    a = ','.join(i)
    file.write(a)
file.close()


In [8]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [9]:
dictionary = corpora.Dictionary(processed_data)

2023-07-06 07:45:19,952 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-07-06 07:45:20,187 : INFO : built Dictionary<15202 unique tokens: ['', '\n', 'A', 'a', 'about']...> from 1000 documents (total 991246 corpus positions)
2023-07-06 07:45:20,188 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15202 unique tokens: ['', '\\n', 'A', 'a', 'about']...> from 1000 documents (total 991246 corpus positions)", 'datetime': '2023-07-06T07:45:20.187850', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'created'}


In [10]:
dictionary.filter_extremes(no_below=2, no_above=0.5)

2023-07-06 07:45:26,485 : INFO : discarding 5480 tokens: [('', 865), ('\n', 1000), ('A', 952), ('a', 1000), ('an', 957), ('and', 999), ('are', 698), ('at', 825), ('between', 648), ('claim', 996)]...
2023-07-06 07:45:26,486 : INFO : keeping 9722 tokens which were in no less than 2 and no more than 500 (=50.0%) documents
2023-07-06 07:45:26,493 : INFO : resulting dictionary: Dictionary<9722 unique tokens: ['about', 'accordance', 'acts', 'angles', 'around']...>


In [11]:
corpus = [dictionary.doc2bow(text) for text in processed_data]

In [18]:
num_topics = 50
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

2023-07-06 07:48:57,791 : INFO : using autotuned alpha, starting with [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
2023-07-06 07:48:57,792 : INFO : using serial LDA version on this node
2023-07-06 07:48:57,820 : INFO : running online (multi-pass) LDA training, 50 topics, 20 passes over the supplied corpus of 1000 documents, updating model once every 1000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2023-07-06 07:48:57,821 : INFO : PROGRESS: pass 0, at document #1000/1000
2023-07-06 07:49:00,540 : INFO : optimized alpha [0.01833965, 0.01767212, 0.017560825, 0.017742515, 0.020452514, 0.017451942, 0.018450048, 0.01792112, 0.017942023, 0.017917315, 0.01807633, 0.018831067, 0.0

2023-07-06 07:49:02,120 : INFO : topic #4 (0.021): 0.122*"said" + 0.020*"region" + 0.016*"according" + 0.014*"device" + 0.012*"semiconductor" + 0.011*"substrate" + 0.010*"system" + 0.008*"surface" + 0.008*"data" + 0.007*"active"
2023-07-06 07:49:02,121 : INFO : topic diff=0.444652, rho=0.447214
2023-07-06 07:49:02,136 : INFO : PROGRESS: pass 4, at document #1000/1000
2023-07-06 07:49:02,545 : INFO : optimized alpha [0.016059097, 0.015298324, 0.014894258, 0.015846632, 0.020815844, 0.015693866, 0.015951378, 0.015515083, 0.015660554, 0.015718078, 0.01602541, 0.016911384, 0.015580874, 0.01499766, 0.014821834, 0.016847325, 0.016506784, 0.016224466, 0.015482079, 0.020292444, 0.016521005, 0.016287982, 0.01923977, 0.017318688, 0.015887743, 0.015059833, 0.014826032, 0.014934062, 0.015359264, 0.01611928, 0.015745245, 0.015849426, 0.019404149, 0.014983424, 0.015801793, 0.015746959, 0.016010333, 0.015451423, 0.016486144, 0.015755357, 0.016051725, 0.016622609, 0.01674078, 0.01676953, 0.0153606655, 

2023-07-06 07:49:03,902 : INFO : PROGRESS: pass 8, at document #1000/1000
2023-07-06 07:49:04,284 : INFO : optimized alpha [0.014893613, 0.014366143, 0.013712462, 0.015199584, 0.020614665, 0.014828629, 0.014653826, 0.014297396, 0.014679013, 0.01474048, 0.014948425, 0.015750708, 0.014354372, 0.014562909, 0.013746841, 0.015610699, 0.01563848, 0.015456471, 0.014397819, 0.022767756, 0.015335767, 0.015345016, 0.01807891, 0.016390115, 0.015927818, 0.013693822, 0.013638804, 0.013583679, 0.014228262, 0.015922312, 0.014351661, 0.015377927, 0.019710407, 0.0141145075, 0.014792208, 0.014752061, 0.01499529, 0.014505088, 0.0160604, 0.014775514, 0.016313724, 0.015445114, 0.016221099, 0.016406534, 0.0150129385, 0.016181163, 0.015329773, 0.015766263, 0.015107254, 0.013856471]
2023-07-06 07:49:04,310 : INFO : topic #27 (0.014): 0.021*"reactive" + 0.018*"substrate" + 0.015*"region" + 0.015*"PD" + 0.014*"active" + 0.013*"accordance" + 0.013*"light-emitting" + 0.011*"tag" + 0.011*"molecule" + 0.010*"LED"
2

2023-07-06 07:49:05,970 : INFO : topic #27 (0.013): 0.021*"reactive" + 0.018*"substrate" + 0.016*"accordance" + 0.015*"region" + 0.015*"PD" + 0.014*"active" + 0.013*"light-emitting" + 0.010*"tag" + 0.010*"LED" + 0.010*"FSQDTs"
2023-07-06 07:49:05,971 : INFO : topic #26 (0.013): 0.037*"sequence" + 0.032*"nucleotide" + 0.021*"detecting" + 0.019*"group" + 0.017*"chain" + 0.016*"said" + 0.016*"selected" + 0.015*"according" + 0.014*"nucleic" + 0.014*"consisting"
2023-07-06 07:49:05,971 : INFO : topic #4 (0.021): 0.153*"said" + 0.020*"region" + 0.019*"according" + 0.015*"system" + 0.014*"device" + 0.009*"surface" + 0.008*"group" + 0.008*"substrate" + 0.008*"programming" + 0.008*"formed"
2023-07-06 07:49:05,972 : INFO : topic #32 (0.021): 0.071*"semiconductor" + 0.055*"device" + 0.028*"according" + 0.026*"said" + 0.019*"forming" + 0.018*"formed" + 0.017*"substrate" + 0.016*"active" + 0.015*"as" + 0.014*"dots"
2023-07-06 07:49:05,972 : INFO : topic #19 (0.026): 0.186*"light" + 0.053*"emitting"

2023-07-06 07:49:07,598 : INFO : topic #4 (0.021): 0.161*"said" + 0.020*"according" + 0.019*"region" + 0.015*"system" + 0.013*"device" + 0.009*"group" + 0.009*"surface" + 0.008*"programming" + 0.008*"link" + 0.008*"data"
2023-07-06 07:49:07,599 : INFO : topic #32 (0.022): 0.078*"semiconductor" + 0.058*"device" + 0.031*"according" + 0.026*"said" + 0.023*"formed" + 0.019*"forming" + 0.017*"substrate" + 0.017*"active" + 0.014*"as" + 0.013*"dots"
2023-07-06 07:49:07,599 : INFO : topic #19 (0.028): 0.194*"light" + 0.054*"emitting" + 0.023*"device" + 0.022*"source" + 0.020*"wavelength" + 0.018*"surface" + 0.015*"LED" + 0.013*"plurality" + 0.012*"according" + 0.011*"display"
2023-07-06 07:49:07,600 : INFO : topic diff=0.120152, rho=0.235702
2023-07-06 07:49:07,615 : INFO : PROGRESS: pass 17, at document #1000/1000
2023-07-06 07:49:07,971 : INFO : optimized alpha [0.013396424, 0.013289193, 0.01227008, 0.014771292, 0.020681707, 0.013706436, 0.0131180575, 0.012769104, 0.013574311, 0.013558916, 0

In [19]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2023-07-06 07:49:08,840 : INFO : CorpusAccumulator accumulated stats from 1000 documents


Average topic coherence: -3.4033.
[([(0.081138596, 'semiconductor'),
   (0.060724672, 'device'),
   (0.031783443, 'according'),
   (0.025914753, 'said'),
   (0.025402356, 'formed'),
   (0.019702196, 'forming'),
   (0.016835786, 'active'),
   (0.016631478, 'substrate'),
   (0.014135516, 'as'),
   (0.013477971, 'layers'),
   (0.013209194, 'barrier'),
   (0.013084084, 'dots'),
   (0.012963396, 'step'),
   (0.012925117, 'structure'),
   (0.012916517, 'electrode'),
   (0.009714933, 'surface'),
   (0.009673773, 'plurality'),
   (0.008796612, 'disposed'),
   (0.008478723, 'insulating'),
   (0.0077507715, 'region')],
  -1.223936885283271),
 ([(0.19832128, 'light'),
   (0.05408073, 'emitting'),
   (0.02291927, 'source'),
   (0.022049973, 'device'),
   (0.020334845, 'wavelength'),
   (0.018492311, 'surface'),
   (0.014734926, 'LED'),
   (0.013027406, 'plurality'),
   (0.011866173, 'according'),
   (0.011002689, 'display'),
   (0.008595126, 'disposed'),
   (0.008515239, 'member'),
   (0.007926001

In [89]:
import numpy as np
topics=[]
for i in top_topics:
    topic = []
#     print(i)
    for j in i:
        topic.append(j)
# #         j = j.astype("int32")
#         for k in range(len(j)):
#             print(j[k][1])
    topics.append(topic)
# topics

In [91]:
import pandas as pd
a = pd.DataFrame(topics)

In [100]:
topics = []
for i in a.iloc[:,0]:
    topic = []
    for j in i:
        topic.append(j[1])
    topics.append(topic)

In [101]:
pd.DataFrame(topics)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,semiconductor,device,according,said,formed,forming,active,substrate,as,layers,barrier,dots,step,structure,electrode,surface,plurality,disposed,insulating,region
1,light,emitting,source,device,wavelength,surface,LED,plurality,according,display,disposed,member,unit,diode,configured,portion,comprisinga,element,including,which
2,plurality,electrode,transport,electrodes,dots,group,charge,device,semiconductor,electrically,nanocrystals,surface,substantially,regions,connected,form,control,formed,carriers,region
3,oxide,structure,silicon,particles,index,device,active,formed,more,storage,region,fiber,forming,transparent,surface,metal,includes,substrate,memory,optical
4,device,nm,dots,providing,doped,wavelength,said,as,light,defined,conversion,range,layers,film,third,photoelectric,optical,semiconductor,than,plurality
5,substrate,forming,silicon,crystal,defect,predetermined,about,film,surface,nm,nanostructure,size,substrate;,pattern,thickness,portion,photonic,sacrificial,device,within
6,semiconductor,C,mixture,alloy,solution,about,formula,has,molecular,°,metal,nanocrystal,dots,B,adding,nanoparticle,which,group,particle,sample
7,energy,device,barrier,optical,band,tunneling,memory,which,electron,than,cavity,formed,well,channel,source,set,forth,gate,photon,said
8,said,radiation,optical,wavelength,element,display,structures,frequency,apparatus,surface,more,elements,laser,output,gain,plurality,conductive,operating,carbon,shape
9,agent,acid,amino,peptide,ID,SEQ,group,consisting,selected,imaging,sequence,NO,moiety,conjugate,chlorotoxin,target,cancer,contrast,canceled,said
